# 1. Imports

In [1]:
from sklearn.decomposition import PCA
import matplotlib as mpl 
import numpy as np
import os
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import f1_score
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import StandardScaler


from utils import stratified_train_test_group_kfold
from utils import model_test_classification

from classification_codes import svc_gridsearch



# 2. Data

In [2]:
with open("Data/MoS2_Analysis_Data_trained2", "rb") as fp:   # Unpickling
    MoS2_Proj1_Class_Data = pickle.load(fp)

#features = MoS2_ImageNet_100_data[0]
df = pd.DataFrame(MoS2_Proj1_Class_Data)
print(df.keys())
T_dict= {900:0, 950:1, 1000:2}
T_target = np.array([T_dict[item] for item in df['T']])

features = np.array(list(df['ImageNet']))
sampleId = np.array(list(df['sampleId']))

X = features
Y = np.array(T_target)
groups = np.array(sampleId)


train_val_groups, train_val_X, train_val_Y, test_X, test_Y = stratified_train_test_group_kfold(X, Y, groups, n_splits=10, test_fold=0)

train_val_Y = train_val_Y.flatten()
test_Y = test_Y.flatten()

print(train_val_X.shape)
print(train_val_Y.shape)
print(test_X.shape)
print(test_Y.shape)


oversample = RandomOverSampler(sampling_strategy='not majority')

scaler = StandardScaler().fit(train_val_X)
train_val_X = scaler.transform(train_val_X)
test_X = scaler.transform(test_X)

Index(['sampleId', 'sampleLabel', 'image', 'T', 'ImageNet', 'MicroNet'], dtype='object')


# 3. Runing Cross-Validation

In [5]:
Ce = [1.0, 0.8, 0.2]
Kernel= ['poly']#, 'rbf', 'sigmoid']#, 'precomputed']
Degree = [i for i in range(2, 8)]
Gamma = ['scale', 'auto', 0.1, 0.2, 0.3, 0.34]# or float, default=’scale’ 0.34 is the best
Coef0 = [0.8, 0.2, 0.05, 0.0]

Max_iter = [90, 100, 120, -1]
variables = [Ce, Kernel, Degree, Gamma, Coef0, Max_iter]


def cross_10_folds_svc(train_val_X, train_val_Y):
    
    best_train = []
    best_val = []
    best_variables = []
    for fold in range(10):
        
        model_path = f'classification/ImageNet/aug4/SVC_{fold}_model.sav'
        
        group, train_X, train_Y, val_X, val_Y = stratified_train_test_group_kfold(train_val_X, train_val_Y, train_val_groups, n_splits=10, test_fold=fold)
        
        #train_X, train_Y = oversample.fit_resample(train_X, train_Y)
        
        variables_best, performance_best = svc_gridsearch(train_X, train_Y, val_X, val_Y, Ce, Kernel, Degree, Gamma, Coef0, Max_iter, fold, model_path)
        best_train.append(performance_best['accuracy_train'])
        best_val.append(performance_best['accuracy_val'])
        best_variables.append(variables_best)
        print(f'fold: {fold} done!')
    return best_train, best_val, best_variables


best_train, best_val, best_variables = cross_10_folds_svc(train_val_X, train_val_Y)
print(best_train)
print(best_val)
print(f'train_acc_mean: {np.mean(best_train) :.3f}, std: {np.std(best_train) :.3f}, val_acc_mean: {np.mean(best_val):.3f}, std: {np.std(best_val):.3f}')


<class 'generator'>
best_train_acc: 0.909952606635071, best_val_acc: 0.7083333333333334
best_train_acc: 0.8720379146919431, best_val_acc: 0.75


/noether/s0/iam5249/anaconda3/envs/torchvision/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=90).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/noether/s0/iam5249/anaconda3/envs/torchvision/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/noether/s0/iam5249/anaconda3/envs/torchvision/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=120).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/noether/s0/iam5249/anaconda3/envs/torchvision/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=90).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn

best_train_acc: 0.8246445497630331, best_val_acc: 0.7916666666666666


/noether/s0/iam5249/anaconda3/envs/torchvision/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/noether/s0/iam5249/anaconda3/envs/torchvision/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=120).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/noether/s0/iam5249/anaconda3/envs/torchvision/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=90).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/noether/s0/iam5249/anaconda3/envs/torchvision/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.war

fold: 0 done!
<class 'generator'>
best_train_acc: 0.9004739336492891, best_val_acc: 0.8333333333333334


...

/noether/s0/iam5249/anaconda3/envs/torchvision/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=120).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/noether/s0/iam5249/anaconda3/envs/torchvision/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=90).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/noether/s0/iam5249/anaconda3/envs/torchvision/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/noether/s0/iam5249/anaconda3/envs/torchvision/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=120).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.war

fold: 8 done!
<class 'generator'>
best_train_acc: 0.9245283018867925, best_val_acc: 0.6521739130434783
best_train_acc: 0.9245283018867925, best_val_acc: 0.6956521739130435
best_train_acc: 0.9245283018867925, best_val_acc: 0.7391304347826086


/noether/s0/iam5249/anaconda3/envs/torchvision/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=90).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/noether/s0/iam5249/anaconda3/envs/torchvision/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/noether/s0/iam5249/anaconda3/envs/torchvision/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=120).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/noether/s0/iam5249/anaconda3/envs/torchvision/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=90).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn

fold: 9 done!
[0.8246445497630331, 0.9620853080568721, 0.8815165876777251, 0.943127962085308, 0.8767772511848341, 0.9339622641509434, 1.0, 0.9622641509433962, 0.8773584905660378, 0.9245283018867925]
[0.7916666666666666, 0.875, 0.7916666666666666, 0.8333333333333334, 0.8333333333333334, 0.9130434782608695, 0.7391304347826086, 0.8695652173913043, 0.6956521739130435, 0.7391304347826086]
train_acc_mean: 0.919, std: 0.050, val_acc_mean: 0.808, std: 0.066


/noether/s0/iam5249/anaconda3/envs/torchvision/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=90).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/noether/s0/iam5249/anaconda3/envs/torchvision/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/noether/s0/iam5249/anaconda3/envs/torchvision/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=120).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/noether/s0/iam5249/anaconda3/envs/torchvision/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=90).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn

In [7]:
import numpy as np
from sklearn.metrics import f1_score

X, Y = train_val_X, train_val_Y
groups = train_val_groups
def f1score_fn(X, Y):

    F1_score_train = []
    F1_score_val = []
    for fold in range(10):
        
        group, train_X, train_Y, val_X, val_Y = stratified_train_test_group_kfold(X, Y, groups, n_splits=10, test_fold=fold)
        #train_X, train_Y = oversample.fit_resample(train_X, train_Y)
        model_path = f'classification/ImageNet/aug4/SVC_{fold}_model.sav'
        
        PATH = os.path.join('Models', model_path)
        
        loaded_model = pickle.load(open(PATH, 'rb'))

        pred_val_Y = loaded_model.predict(val_X)
        pred_train_Y = loaded_model.predict(train_X)

        f_score_train = f1_score(train_Y, pred_train_Y, average='macro')#'weighted')
        f_score_val = f1_score(val_Y, pred_val_Y, average='macro')#'weighted')

        F1_score_train.append(f_score_train)
        F1_score_val.append(f_score_val)
    
    return F1_score_train, F1_score_val



F1_score_train, F1_score_val = f1score_fn(X, Y)


print(f'f1_train: {np.mean(F1_score_train)}, std: {np.std(F1_score_train)}, f1_val: {np.mean(F1_score_val)}, std: {np.std(F1_score_val)}')


<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
f1_train: 0.9055980491662267, std: 0.059353635312323204, f1_val: 0.7113023744256546, std: 0.08941465970995119
